In [2]:
%%time
from app.database import Database
from app.classes import IterationStatus, BatchAttemptStatus, TestStatus, AttemptStatus
from typing import List

db = Database()
# batch_li = [x for x in db.get_all() if x.dataset == 'commons-csv']
all_dataset = db.get_all()
for d in all_dataset:
    print(d.doc_id, d.dataset)
    if d.doc_id in [82, 101, 123]:
        print("=============")

read time: 0.03900003433227539
decompress time: 3.1128597259521484
loads time: 3.5241565704345703
assembling time: 3.1167263984680176
47 lang_1_fixed
58 commons-csv
61 commons-cli
66 gson
82 jfreechart
89 lang_1_fixed
98 gson
99 commons-csv
100 commons-cli
101 jfreechart
123 jfreechart154
126 jfreechart154
127 lang_1_fixed
128 gson
130 jfreechart154
CPU times: total: 12.1 s
Wall time: 12.2 s


In [3]:
from tqdm import tqdm

batch_li = [x for x in all_dataset if x.doc_id in [47, 58, 61, 66, 123]]
all_attempts = sum([x.attempts for x in batch_li], [])
print(len(all_attempts))

8192


将这些Java编译错误进行归类需要考虑它们出现的上下文和原因。许多错误可能在某些情况下同时发生，但并不意味着它们总是紧密相关或一个错误的出现必然导致另一个。以下是对这些错误的分析，尝试将它们根据相似的原因或上下文归类：

1. **关于类和方法定义的错误**:
   - "Illegal static declaration in inner class"
   - "cannot inherit from final"
   - "method does not override or implement a method from a supertype"
   - "cannot override"
   - "is not abstract and does not override abstract method"
   - "enum types must not be local"
   - "is abstract; cannot be instantiated"

2. **关于变量和类型的错误**:
   - "cannot assign a value to final variable"
   - "cannot infer type arguments"
   - "actual and formal argument lists differ in length"
   - "must be final or effectively final"
   - "type not allowed here"
   - "is not within bounds of"
   - "incompatible types"
   - "cannot be dereferenced"

3. **关于构造函数和实例化的错误**:
   - "no suitable constructor found"
   - "qualified new of static class"
   - "enum types may not be instantiated"

4. **关于访问修饰符和文件结构的错误**:
   - "is public, should be declared in a file named"

5. **关于异常处理的错误**:
   - "unreported exception"
   - "is never thrown in body of corresponding try statement"

6. **关于循环和迭代的错误**:
   - "for-each not applicable to expression type"

7. **关于方法调用和参数的错误**:
   - "no suitable method found for"
   - "cannot be applied to given types"

8. **关于语法和表达式的错误**:
   - "cannot find symbol"
   - "illegal start of expression"
   - "unclosed character literal"
   - "illegal escape character"

9. **关于重复定义和命名的错误**:
   - "is already defined in"

10. **关于模糊性和异常的错误**:
    - "is ambiguous"
    - "has already been caught"
    - "an enclosing instance that contains"

11. **关于枚举和实例化的错误**:
    - "enum types may not be instantiated"

12. **关于数值和字符字面量的错误**:
    - "integer number too large"

尽管某些错误可能在某些特定情况下同时发生，大多数错误类型都是因为不同的编程错误而独立出现的。因此，它们大多数可以被视为单独的类别，但上述归类尝试根据错误的性质和它们可能共享的上下文将它们分组。

In [236]:


def get_error_msg(attempt):
    import requests
    base_url = "http://127.0.0.1:4200/api/logs/filter"
    payload = {
        "logs": {
            "level": {
                "ge_": 40
            },
            "flow_run_id": {
                "any_": [
                    f"{attempt.link.split('/')[-1]}"
                ]
            }
        },
        "sort": "TIMESTAMP_ASC",
        "offset": 0,
        "limit": 200
    }

    # parse response
    response = requests.post(base_url, json=payload)
    arr = response.json()
    messages = [x['message'] for x in arr]
    return messages


error_types = [
    "cannot find symbol",
    (["cannot be accessed from outside package","cannot access","has private access","has protected access",], "cannot access"),

    "is ambiguous",
    "incompatible types",
    "unclosed character literal",
    "illegal escape character",
    "is not abstract and does not override abstract method",
    "is already defined in",  # system error
    "unreported exception",
    "integer number too large",  # system error
    "illegal start of expression",
    "cannot be applied to given types",
    "enum types must not be local",
    "cannot be dereferenced",
    "is abstract; cannot be instantiated",
    "does not exist",
    "is never thrown in body of corresponding try statement",
    (["reached end of file while parsing","not a statement",r"\';\' expected", r"\')\' expected", r"')' expected", r"'\' expected", r"';' expected", r"\';\' expected"],
     "syntax error"),
    "Illegal static declaration in inner class",
    "cannot assign a value to final variable",
    "cannot infer type arguments",
    "cannot inherit from final",
    "actual and formal argument lists differ in length",
    (["no suitable constructor found","no suitable method found for",], "no suitable constructor/method found"),
    "is public, should be declared in a file named",
    "qualified new of static class",
    "method does not override or implement a method from a supertype",
    "must be final or effectively final",
    "cannot override",
    "type not allowed here",
    "has already been caught",  # system error
    "an enclosing instance that contains",
    "enum types may not be instantiated",
    "is not within bounds of",
    "for-each not applicable to expression type",
]


def get_error_set(msg_list, status):
    msg_list = [
        x for x in msg_list
        if "[ERROR] COMPILATION ERROR" in x
    ]
    error_set = {status}
    for msg in msg_list:
        flag = False
        for error_type in error_types:
            if isinstance(error_type, tuple):
                for sub_error_type in error_type[0]:
                    if sub_error_type in msg:
                        flag = True
                        error_set.add(error_type[1])
            elif error_type in msg:
                flag = True
                error_set.add(error_type)
        if not flag:
            print(msg)
            assert False
    if status == str(IterationStatus.Type.SYNTAX_ERROR):
        error_set.add("syntax error")
        # if flag:
        #     break
    return error_set

In [219]:
count = 0
attempts = all_attempts

In [215]:
from tqdm import tqdm
message_list = []
for index in tqdm(range(count, len(attempts))):
    messages = get_error_msg(attempts[index])
    message_list.append(messages)

100%|██████████| 8192/8192 [01:40<00:00, 81.19it/s] 


In [237]:
# 统计错误类型
from collections import Counter

error_sets = []
for index in range(0, len(message_list)):
    attempt = attempts[index]
    if attempt.type == IterationStatus.Type.PASS or attempt.type == IterationStatus.Type.RUNTIME_ERROR:
        if attempt.index != len(attempt.sub_iteration_status):
            for it in attempt.sub_iteration_status[attempt.index:]:
                if it.type == IterationStatus.Type.COMPILE_ERROR:
                    continue
    error_sets.append(get_error_set(message_list[index], str(attempt.type)))
    count += 1

error_counter = Counter()
PASS_error_sets = []
COMPILE_ERROR_error_sets = []
RUNTIME_ERROR_error_sets = []
SYNTAX_ERROR_error_sets = []
for error_set in error_sets:
    if str(IterationStatus.Type.COMPILE_ERROR) in error_set:
        error_set.remove(str(IterationStatus.Type.COMPILE_ERROR))
        COMPILE_ERROR_error_sets.append(error_set)
    if str(IterationStatus.Type.PASS) in error_set:
        error_set.remove(str(IterationStatus.Type.PASS))
        PASS_error_sets.append(error_set)
    if str(IterationStatus.Type.RUNTIME_ERROR) in error_set:
        error_set.remove(str(IterationStatus.Type.RUNTIME_ERROR))
        RUNTIME_ERROR_error_sets.append(error_set)
    if str(IterationStatus.Type.SYNTAX_ERROR) in error_set:
        error_set.remove(str(IterationStatus.Type.SYNTAX_ERROR))
        SYNTAX_ERROR_error_sets.append(error_set)
    error_counter.update(error_set)


# 删除系统错误
remove_list = [
    "is already defined in",
    "integer number too large",
    "has already been caught",
]
while True:
    for x in remove_list:
        if x in error_counter:
            del error_counter[x]
            break
    else:
        break

# 将 <50 的类型合并为 "others"
ec = Counter()
for error_set in error_sets:
    error_set = [x if error_counter[x] > 50 else "others" for x in error_set]
    ec.update(error_set)

pass_ec = Counter()
for error_set in sum([PASS_error_sets,RUNTIME_ERROR_error_sets],[]):
    error_set = [x if error_counter[x] > 50 else "others" for x in error_set]
    pass_ec.update(error_set)



In [238]:
import pandas as pd

# Convert the Counter objects to dictionaries
ec_dict = dict(ec)
pass_ec_dict = dict(pass_ec)

# Merge the two dictionaries into a new dictionary
merged_dict = {key: [ec_dict.get(key, 0), pass_ec_dict.get(key, 0), ec_dict.get(key, 0) - pass_ec_dict.get(key, 0)] for key in set(ec_dict) | set(pass_ec_dict)}

# Convert the merged dictionary to a pandas DataFrame
df = pd.DataFrame.from_dict(merged_dict, orient='index', columns=['ec', 'pass_ec', 'diff'])
# Sort the DataFrame by 'ec' in descending order
df = df.sort_values(by='ec', ascending=False)

# Save the DataFrame to a CSV file
df.to_csv('编译错误类型统计.csv', encoding='utf_8_sig')

In [44]:
# import matplotlib.pyplot as plt
#
# def pie_counter(counter):
#     labels = counter.keys()
#     sizes = counter.values()
#
#     fig1, ax1 = plt.subplots(dpi=300)
#     ax1.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90)
#     ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
#     plt.savefig("error_type_pie.png")
#     plt.show()
# pie_counter(ec)

In [52]:
# 运行错误统计
# 类型一： 模版修复成功。找auto_fix=True, try to fix 1 failures and 0 errors!
# 类型二： 模版修复失败。
# 类型三： 大模型修复成功。 token >= 2
# 类型四： 大模型修复失败。
def get_all_msg(attempt):
    import requests
    base_url = "http://127.0.0.1:4200/api/logs/filter"
    payload = {
        "logs": {
            "level": {
                "ge_": 20
            },
            "flow_run_id": {
                "any_": [
                    f"{attempt.link.split('/')[-1]}"
                ]
            }
        },
        "sort": "TIMESTAMP_ASC",
        "offset": 0,
        "limit": 200
    }

    # parse response
    response = requests.post(base_url, json=payload)
    arr = response.json()
    messages = [x['message'] for x in arr if 40 > x['level'] >= 20]
    return messages

from tqdm import tqdm
all_message_list = []
for index in tqdm(range(0, len(attempts))):
    messages = get_all_msg(attempts[index])
    all_message_list.append(messages)

100%|██████████| 8192/8192 [02:59<00:00, 45.63it/s]


In [209]:
failure_types = [
    "org.junit.Assert.assertEquals",
    (["org.junit.Assert.assertTrue", "org.junit.Assert.assertFalse"], "assertTrue/False"),
    (["org.junit.Assert.assertNotNull", "org.junit.Assert.assertNull"], "assertNull/NotNull"),
    (["org.mockito.exceptions","Argument(s) are different!","Wanted but not invoked"], "mockito"),
    "org.junit.Assert.assertThat",
]


def get_failure_set(msg_list, status):
    msg_list = [
        x for x in msg_list
        if "[INFO]  T E S T S" in x
    ]
    pattern = r"Tests run: (\d+), Failures: (\d+), Errors: (\d+)"
    import re
    def get_exec_tuple(message: str):
        match = re.search(pattern, message)
        if match:
            return tuple(map(int, match.groups()))
        return (0, 0, 0)
    failure_set = {status}
    for msg in msg_list:
        t = get_exec_tuple(msg)
        if t[1] + t[2] == 0:
            continue
        flag = False
        if t[2] > 0:
            failure_set.add("error")
            continue
        for failure_type in failure_types:
            if isinstance(failure_type, tuple):
                for sub_failure_type in failure_type[0]:
                    if sub_failure_type in msg:
                        flag = True
                        failure_set.add(sub_failure_type)
            elif failure_type in msg:
                flag = True
                failure_set.add(failure_type)
        if not flag:
            failure_set.add("fail" if "org.junit.Assert.fail" in msg else "others")


    return failure_set

In [210]:
# 统计错误类型
from collections import Counter

failure_sets = []
skip = 0
for index in tqdm(range(0, len(all_message_list))):
    attempt = attempts[index]
    if attempt.type == IterationStatus.Type.PASS:
        if attempt.index != len(attempt.sub_iteration_status):
            for it in attempt.sub_iteration_status[attempt.index:]:
                if it.type == IterationStatus.Type.RUNTIME_ERROR:
                    skip += 1
                    continue
    failure_sets.append(get_failure_set(all_message_list[index], str(attempt.type)))
    count += 1
print(skip)
failure_counter = Counter()
PASS_failure_sets = []
COMPILE_ERROR_failure_sets = []
RUNTIME_ERROR_failure_sets = []
SYNTAX_ERROR_failure_sets = []
filtered_failure_sets = []
for failure_set in failure_sets:
    if str(IterationStatus.Type.COMPILE_ERROR) in failure_set:
        continue
        failure_set.remove(str(IterationStatus.Type.COMPILE_ERROR))
        COMPILE_ERROR_failure_sets.append(failure_set)
    if str(IterationStatus.Type.PASS) in failure_set:
        failure_set.remove(str(IterationStatus.Type.PASS))
        PASS_failure_sets.append(failure_set)
    if str(IterationStatus.Type.RUNTIME_ERROR) in failure_set:
        failure_set.remove(str(IterationStatus.Type.RUNTIME_ERROR))
        RUNTIME_ERROR_failure_sets.append(failure_set)
    if str(IterationStatus.Type.SYNTAX_ERROR) in failure_set:
        continue
        failure_set.remove(str(IterationStatus.Type.SYNTAX_ERROR))
        SYNTAX_ERROR_failure_sets.append(failure_set)
        continue
    if str(IterationStatus.Type.FAIL) in failure_set:
        continue
    filtered_failure_sets.append(failure_set)
    failure_counter.update(failure_set)


# 删除系统错误
remove_list = [
    "is already defined in",
    "integer number too large",
    "has already been caught",
]
while True:
    for x in remove_list:
        if x in failure_counter:
            del failure_counter[x]
            break
    else:
        break

# 将 <50 的类型合并为 "others"
ec = Counter()
for failure_set in filtered_failure_sets:
    failure_set = [x if failure_counter[x] > 50 else "others" for x in failure_set]
    ec.update(failure_set)

pass_ec = Counter()
for failure_set in PASS_failure_sets:
    failure_set = [x if failure_counter[x] > 50 else "others" for x in failure_set]
    pass_ec.update(failure_set)

print(failure_counter)
ec

100%|██████████| 8192/8192 [00:00<00:00, 26757.19it/s]

126
Counter({'org.junit.Assert.assertEquals': 1360, 'error': 918, 'org.junit.Assert.assertTrue': 436, 'fail': 206, 'org.junit.Assert.assertFalse': 178, 'org.junit.Assert.assertNotNull': 117, 'org.junit.Assert.assertNull': 108, 'Wanted but not invoked': 30, 'others': 18, 'Argument(s) are different!': 7, 'org.mockito.exceptions': 6, 'org.junit.Assert.assertThat': 2})


Counter({'org.junit.Assert.assertEquals': 1360,
         'error': 918,
         'org.junit.Assert.assertTrue': 436,
         'fail': 206,
         'org.junit.Assert.assertFalse': 178,
         'org.junit.Assert.assertNotNull': 117,
         'org.junit.Assert.assertNull': 108,
         'others': 63})

In [211]:
import pandas as pd

# Convert the Counter objects to dictionaries
ec_dict = dict(ec)
pass_ec_dict = dict(pass_ec)

# Merge the two dictionaries into a new dictionary
merged_dict = {key: [ec_dict.get(key, 0), pass_ec_dict.get(key, 0), ec_dict.get(key, 0) - pass_ec_dict.get(key, 0)] for key in set(ec_dict) | set(pass_ec_dict)}

# Convert the merged dictionary to a pandas DataFrame
df = pd.DataFrame.from_dict(merged_dict, orient='index', columns=['ec', 'pass_ec', 'diff'])
# Sort the DataFrame by 'ec' in descending order
df = df.sort_values(by='ec', ascending=False)

# Save the DataFrame to a CSV file
df.to_csv('运行错误类型统计.csv', encoding='utf_8_sig')